- [Import Libraries](#1)
- [Reading the Dataset](#2)
- [Data Pre-processing](#3)
- [Modeling](#4)
    1. [Simple CNN](#8)
    2. [Transform learning with MobileNetV2](#9)
- [Conclusions](#5)
- [Gradio ](#6)
- [References](#7)

# Import Libraries <a id = "1"></a> 

In [1]:
!pip install -q gradio

In [2]:
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras.applications.mobilenet_v2 import preprocess_input, MobileNetV2
from keras.layers import BatchNormalization, Conv2D, Dense,Dropout, Flatten, GlobalAveragePooling2D, MaxPool2D, RandomFlip, RandomRotation, ReLU, Rescaling

import warnings
warnings.filterwarnings("ignore")

# Reading the Dataset <a id = "2"></a> 

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

<a href = 'https://www.cs.toronto.edu/~kriz/cifar.html'>Link to CIFAR dataset.</a>

<i>"The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.<br>
The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class."</i>

In [4]:
print(f'Shape of training data: {x_train.shape}')
print(f'Shape of training labels: {y_train.shape}')
print(f'Number of training samples: {x_train.shape[0]}')
print(15 * '-')
print(f'Shape of testing data: {x_test.shape}')
print(f'Shape of testing labels: {y_test.shape}')
print(f'Number of testing samples: {x_test.shape[0]}')
print(15 * '-')
print(f'Size of images: {x_train.shape[1:4]}')

# Data Pre-processing <a id = "3"></a> 

In [5]:
sns.countplot(np.squeeze(y_train))
plt.xlabel('Index of each class')

In [6]:
index_to_name = {0:'Airplane', 1:'Car', 2:'Bird',
                 3:'Cat', 4:'Deer', 5:'Dog',
                 6:'Frog', 7:'Horse', 8:'Ship',
                 9:'Truck'}

In [7]:
plt.figure(figsize = (15, 9))
for num, i in enumerate(np.random.randint(x_train.shape[0],size = 9)):
    plt.subplot(3,3, num + 1)
    plt.imshow(x_train[i])
    class_index = np.squeeze(y_train[i][0]).astype(int)
    plt.title(f'{class_index}: {index_to_name[class_index]}')
    plt.axis('off')

# Modeling <a id = "4"></a> 


## Simple CNN <a id = "8"></a> 

In [8]:
def make_base_model():
    inputs = keras.Input(shape = (32, 32, 3)) 
    x = Rescaling(1. / 255)(inputs)
    x = RandomFlip()(x)
    x = RandomRotation(0.2, fill_mode = 'nearest')(x)
    x = Conv2D(filters = 8, kernel_size =  2, strides = 2)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis = -1)(x)
    x = MaxPool2D(pool_size = 2, strides = 2)(x)
    x = ReLU()(x)
    x = Conv2D(filters = 16, kernel_size =  2, strides = 2, padding = 'same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis = -1)(x)
    x = MaxPool2D(pool_size = 4, strides = 4)(x)
    x = Flatten()(x)
    x = Dropout(0.2)(x)
    outputs = Dense(10, activation = 'softmax')(x)
    model = keras.Model(inputs = inputs, outputs = outputs, name = 'base_model')
    
    return model   

base_model = make_base_model()
base_model.summary()

In [9]:
base_model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
    filepath = 'best_base_model.keras',
    save_best_only = True,
    monitor = 'val_accuracy')
]

epochs = 20

history = base_model.fit(
    x_train,
    y_train,
    batch_size = 32,
    epochs = epochs,
    shuffle = True,
    validation_split= 0.2,
    callbacks = callbacks
)

In [10]:
def plot_history(history_of_model):
    plt.figure(figsize = (20, 5))
    
    plt.subplot(1, 2, 1)
    plt.title('Loss on training and validation data')
    sns.lineplot(x = range(1, epochs + 1), y = history_of_model.history['loss'])
    sns.lineplot(x = range(1, epochs + 1), y = history_of_model.history['val_loss'])
    plt.xlabel('epochs')
    plt.xticks(list(range(1, epochs + 1))[::2])
    plt.legend(['training loss', 'validation loss'], loc = 'upper left')
    
    plt.subplot(1, 2, 2)
    plt.title('Accuracy on training and validation data')
    sns.lineplot(x = range(1, epochs + 1), y = history_of_model.history['accuracy'])
    sns.lineplot(x = range(1, epochs + 1), y = history_of_model.history['val_accuracy'])
    plt.xlabel('epochs')
    plt.xticks(list(range(1, epochs + 1))[::2])
    plt.legend(['training accuracy', 'validation accuracy'], loc = 'upper left')
    
    plt.show() 
    
plot_history(history)    

In [11]:
best_base_model = keras.models.load_model('best_base_model.keras')
test_loss, test_acc = best_base_model.evaluate(x_test, y_test)
print(f'Accuracy on testing data with a simple CNN model: {test_acc:0.2%}')

In [12]:
def plot_prob_pred(model, x):

    plt.figure(figsize = (20, 8))
    
    m = x_test.shape[0]
    
    for num, i in enumerate(np.random.randint(m, size = 9)):
            
        pred_prob = model.predict(x[i][np.newaxis, ...])
        pred_class = np.argmax(pred_prob).astype(int)

        plt.subplot(3,6, 2 * num + 1)
        plt.imshow(x_test[i])
        plt.title(f'Prediction--> {pred_class}: {index_to_name[pred_class]}\nGroundTruth--> {y_test[i][0]}: {index_to_name[y_test[i][0]]}')
        plt.axis('off')

        plt.subplot(3,6, 2 * num + 2)
        bar_plot = plt.bar(x = range(10), height = list(np.squeeze(pred_prob)))
        bar_plot[pred_class].set_color('red')
        bar_plot[y_test[i][0]].set_color('green')
        plt.xticks(range(10))
        plt.ylim([0, 1])
    
    plt.tight_layout()

plot_prob_pred(best_base_model, x_test)

## Transform learning with MobileNetV2 <a id = "9"></a> 

In [13]:
base_mobilenet = MobileNetV2(input_shape = (32, 32, 3), weights = 'imagenet', include_top = False)

x_train_pr = preprocess_input(x_train)
x_test_pr = preprocess_input(x_test)

In [14]:
x = base_mobilenet.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation = 'softmax')(x)
x = Dropout(0.5)(x)
output_mobilenet = Dense(10, activation = 'softmax')(x)

mobilenetv2 = keras.Model(inputs = base_mobilenet.inputs, outputs = output_mobilenet, name = 'base_mobilenet_model')

mobilenetv2.summary()

In [15]:
for i, layer in enumerate(mobilenetv2.layers):
    print(f'Layer number {i}: {layer.name}')

In [16]:
for layer in mobilenetv2.layers[:134]:
    layer.trainable = False
    
for layer in mobilenetv2.layers[134:]:
    layer.trainable = True

In [17]:
mobilenetv2.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

callbacks_mobilenet = [
    keras.callbacks.ModelCheckpoint(
    filepath = 'best_mobilenet_model.keras',
    save_best_only = True,
    monitor = 'val_accuracy')
]

history_mobilenet = mobilenetv2.fit(
    x_train_pr,
    y_train,
    batch_size = 32,
    epochs = epochs,
    shuffle = True,
    validation_split= 0.2,
    callbacks = callbacks_mobilenet
)

In [18]:
plot_history(history_mobilenet)

In [19]:
best_mobilenet_model = keras.models.load_model('best_mobilenet_model.keras')
test_mobilenet_loss, test_mobilenet_acc = best_mobilenet_model.evaluate(x_test_pr, y_test)
print(f'Accuracy on testing data with a transformed MobileNetV2 model: {test_mobilenet_acc:0.2%}')

In [20]:
plot_prob_pred(best_mobilenet_model, x_test_pr)

# Conclusions <a id = "5"></a> 

Due to the low quality of the photos, the **optimal error rate** is assumed to be 90%.
The CNN model had many **avoidable biases**. This problem can be solved by increasing the model size or by changing the model architecture. Due to this, the pre-trained MobileNetV2 was used.
Accordingly, the change in the model architecture significantly reduced **avoidable bias**.To achieve a better result, you can increase the epochs or start training from the basic layers of the mobile model. Increasing the epochs or starting from the basics of the mobilenetv2 model will allow you to achieve a better result.
<br>

<p style="text-align:center;"><img src="https://miro.medium.com/max/1400/1*dPsfWuAvNJm29val0Uek9w.png" width="450"></p>

# Gradio <a id = "6"></a> 

In [21]:
def predict_with_mobilenetv2(inp):
    inp_pr = preprocess_input(inp[np.newaxis, ...])
    pred_prob = best_mobilenet_model.predict(inp_pr).flatten().tolist()
    return {index_to_name[i]: pred_prob[i] for i in range(10)}

In [ ]:
iface = gr.Interface(fn = predict_with_mobilenetv2,
                     inputs = gr.inputs.Image(shape = (32, 32)),
                     outputs = gr.outputs.Label(num_top_classes = 5),
                     live = True
                    ).launch(share = True, debug = True)

# References <a id = "7"></a> 

- <a href = 'https://towardsdatascience.com/two-important-machine-learning-concepts-to-improve-every-model-62fd058916b'>Bias and Variance: Two Important Machine Learning Concepts to Improve Every Model</a>